# Noughts and crosses

Given a Tic Tac Toe grid in 3 dimension that we want to fill with 14 white balls and 13 black balls.
This N&C model consist on finding the best combinaison of the 14 white balls to have the minimum number of white straight lines.
 

In [ ]:
!pip install docplex==0.0.24

In [ ]:
from docplex.mp.model import Model

Firstly we create the 49 lines and the 27 cells of our cube.

In [ ]:
r =range(1,4)

cell= [(r1,r2,r3) for r1 in r for r2 in r for r3 in r]

lines=[]


for x in [(x1, y1, z1, x2, y2, z2, x3, y3, z3) for x1 in r for x2 in r for x3 in r for y1 in r for y2 in r for y3 in r for z1 in r for z2 in r for z3 in r if (x2-x1==x3-x2) if (y2-y1==y3-y2) if (z2-z1==z3-z2) if (9*x1+3*y1+z1<9*x3+3*y3+z3) ]:
    lines.append(x)

assert len(lines)==49;

In [ ]:
from docplex.mp.environment import Environment
env = Environment()
env.print_information()

We now create our model and solve it

In [ ]:
from docplex.mp.context import DOcloudContext
context = DOcloudContext(url="PUT_YOUR_DOCLOUD_URL_HERE", 
                         api_key="PUT_YOUR_DOCLOUD_KEY_HERE")

In [ ]:
mdl = Model(docloud_context=context)

lines_monocolor = mdl.binary_var_dict(lines, name="is_monocolor")

cell_dict=  mdl.binary_var_dict(cell, name="is_cell")


for l in lines:
    ind1 = mdl.add_indicator(lines_monocolor[l], cell_dict[l[0],l[1],l[2]]+ cell_dict[l[3],l[4],l[5]]+cell_dict[l[6],l[7],l[8]] >=1 , active_value=0 )
    
    ind2 = mdl.add_indicator(lines_monocolor[l], cell_dict[l[0],l[1],l[2]]+ cell_dict[l[3],l[4],l[5]]+cell_dict[l[6],l[7],l[8]] <=2 , active_value=0 )

mdl.add_constraint(mdl.sum(cell_dict[h]  for h in cell) == 14)


mdl.print_information()

mdl.minimize(mdl.sum(lines_monocolor[l] for l in lines))

ok = mdl.solve()

if not ok:   
     print("N&C model fails")
else:
     mdl.print_solution()

In [ ]:
env = mdl.environment

In [ ]:
# Print of the solution
print ("##############Diff print##############")
monocolor_lines_position = []
for l in lines:
    monocolor_lines_position.append( int(float(lines_monocolor[l])))

print(monocolor_lines_position)

game_cells = []
for l in cell:
    game_cells.append(float(cell_dict[l]))

if env.has_numpy:
    import numpy as np
    game_cells = np.reshape(game_cells, (-1,3))
else:
    ii = len(game_cells) // 3
    game_cells2 = [[0 for j in range(0, 3)] for i in range(0, ii)]
    for i in range(0, ii):
        for j in range(0, 3):
            game_cells2[i][j] = game_cells[i * 3 + j]
    game_cells = game_cells2

print(game_cells)

print ("total lines :{0!s} " .format(int(mdl.objective_value)) )

We represent the results on a 3d grid

In [ ]:
monocolor_lines_points = []
for l in lines:
    if float(lines_monocolor[l]) != 0: 
        monocolor_lines_points.append( l)


k2=5   # Space the graph
k1=k2/2

xs=[]
ys=[]
zs=[]
VecStart_xs = []
VecStart_ys = []
VecStart_zs = []

VecEnd_xs = []
VecEnd_ys = []
VecEnd_zs  =[]

for i in range(len(monocolor_lines_points)):
        xs.append(monocolor_lines_points[i][0])
        VecStart_xs.append(monocolor_lines_points[i][0])
        ys.append(monocolor_lines_points[i][1])
        VecStart_ys.append(monocolor_lines_points[i][1])
        
        if monocolor_lines_points[i][2] == 1:
            zs.append(monocolor_lines_points[i][2])
            VecStart_zs.append(monocolor_lines_points[i][2])
        elif monocolor_lines_points[i][2] == 2:
            zs.append(k1+monocolor_lines_points[i][2])
            VecStart_zs.append(k1+monocolor_lines_points[i][2])
        else :
            zs.append(k2+monocolor_lines_points[i][2])
            VecStart_zs.append(k2+monocolor_lines_points[i][2])
            
        xs.append(monocolor_lines_points[i][3])
        ys.append(monocolor_lines_points[i][4])
        
        if monocolor_lines_points[i][5] == 1:
            zs.append(monocolor_lines_points[i][5])
        elif monocolor_lines_points[i][5] == 2:
            zs.append(k1+monocolor_lines_points[i][5])
        else :
            zs.append(k2+monocolor_lines_points[i][5])
            
        
        xs.append(monocolor_lines_points[i][6])
        VecEnd_xs.append(monocolor_lines_points[i][6])
        ys.append(monocolor_lines_points[i][7])
        VecEnd_ys.append(monocolor_lines_points[i][7])
        
        if monocolor_lines_points[i][8] == 1:
            zs.append(monocolor_lines_points[i][8])
            VecEnd_zs.append(monocolor_lines_points[i][8])
        elif monocolor_lines_points[i][8] ==2:
            VecEnd_zs.append(k1+monocolor_lines_points[i][8])
            zs.append(k1+monocolor_lines_points[i][8])
        else :
            zs.append(k2+monocolor_lines_points[i][8])
            VecEnd_zs.append(k2+monocolor_lines_points[i][8])

You can uncomment the 3rd line if you want an inline graphic and not a pop up

In [ ]:
if env.has_matplotlib:
    from mpl_toolkits.mplot3d import Axes3D
    import matplotlib.pyplot as plt
    %matplotlib inline

In [ ]:
if env.has_matplotlib:
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    ax.scatter(xs, ys, zs, c='r', marker='o', s=100)    

    x1=[3,3,1.5,2.5,3,3,1.5,2.5,3,3,1.5,2.5]
    y1=[1.5,2.5,1,1,1.5,2.5,1,1,1.5,2.5,1,1]
    z1=[3+k2,3+k2,3+k2,3+k2,2+k1,2+k1,2+k1,2+k1,1,1,1,1]

    x2=[1,1,1.5,2.5,1,1,1.5,2.5,1,1,1.5,2.5]
    y2=[1.5,2.5,3,3,1.5,2.5,3,3,1.5,2.5,3,3]
    z2=[3+k2,3+k2,3+k2,3+k2,2+k1,2+k1,2+k1,2+k1,1,1,1,1]

    for i in range(len(x1)):

        VecStart_x = [x1[i]]
        VecStart_y = [y1[i]]
        VecStart_z = [z1[i]]

        VecEnd_x = [x2[i]]
        VecEnd_y = [y2[i]]
        VecEnd_z  =[z2[i]]

        ax.plot(VecStart_x + VecEnd_x, VecStart_y + VecEnd_y, VecStart_z +VecEnd_z, c='b')

    for i in range(len(VecStart_xs)):

        VecStart_x = [VecStart_xs[i]]
        VecStart_y = [VecStart_ys[i]]
        VecStart_z = [VecStart_zs[i]]

        VecEnd_x = [VecEnd_xs[i]]
        VecEnd_y = [VecEnd_ys[i]]
        VecEnd_z  =[VecEnd_zs[i]]

        ax.plot(VecStart_x + VecEnd_x, VecStart_y + VecEnd_y, VecStart_z +VecEnd_z, c='r')


    ax.set_xlabel('X Label')
    ax.set_ylabel('Y Label')
    ax.set_zlabel('Z Label')

    plt.show()